In [1]:
import torch
import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 20
batch_size = 64
lr = 1e-3

In [3]:
device

device(type='cuda')

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
train_data = datasets.CIFAR10("data/", train=True, download=True, transform=transform)
val_data = datasets.CIFAR10("data/", train=False, download=True, transform=transform)

In [6]:
train_batches = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_batches = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [7]:
len(train_data.classes)

10

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3, 3))
        self.fc1 = nn.Linear(32 * 30 * 30, 10)

        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.dropout(self.conv1(x))
        x = self.relu(x)

        x = x.view(x.shape[0], -1)
        x = self.fc1(x)

        return x

In [9]:
net = Net().to(device)

In [10]:
inp = torch.randn(1, 3, 32, 32).to(device)
output = net(inp)
output.shape

torch.Size([1, 10])

In [11]:
num_parameters = sum(p.numel() for p in net.parameters() if p.requires_grad)
num_parameters

288906

In [12]:
opt = torch.optim.Adam(net.parameters(), lr)
loss_fn = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

In [13]:
def get_accuracy(preds, y):
    preds = preds.argmax(dim=1, keepdim=True)
    correct = preds.squeeze(1).eq(y)
    acc = correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)

    return acc.item()

In [14]:
def loop(net, batches, train):
    batch_losses = []
    batch_accs = []

    if train:
        print("Train Loop:")
        print("")
        net.train()

        for X, y in tqdm.tqdm(batches, total=len(batches)):
            X = X.to(device)
            y = y.to(device)


            with torch.cuda.amp.autocast():
                preds = net(X)
                loss = loss_fn(preds, y)
                acc = get_accuracy(preds, y)

            opt.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

            batch_losses.append(loss.item())
            batch_accs.append(acc)

    else:
        print("Validation Loop:")
        print("")
        net.eval()

        with torch.no_grad():
            for X, y in tqdm.tqdm(batches, total=len(batches)):
                X = X.to(device)
                y = y.to(device)

                preds = net(X)
                loss = loss_fn(preds, y)
                acc = get_accuracy(preds, y)

                batch_losses.append(loss.item())
                batch_accs.append(acc) 

    print("")
    print("")
    
    return sum(batch_losses) / len(batch_losses), sum(batch_accs) / len(batch_accs)

In [15]:
for epoch in range(epochs):
    train_loss, train_acc = loop(net, train_batches, True)
    val_loss, val_acc = loop(net, val_batches, False)

    print(f"epoch: {epoch} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} | val_acc: {val_acc:.4f}")
    print("")

  0%|          | 0/782 [00:00<?, ?it/s]

Train Loop:



  9%|▉         | 14/157 [00:00<00:01, 138.95it/s]



Validation Loop:



  1%|▏         | 11/782 [00:00<00:07, 105.35it/s]

...



epoch: 18 | train_loss: 0.8349 | train_acc: 0.7070 | val_loss: 1.1629 | val_acc: 0.6064

Train Loop:



  9%|▉         | 14/157 [00:00<00:01, 138.05it/s]



Validation Loop:



100%|██████████| 157/157 [00:01<00:00, 140.42it/s]



epoch: 19 | train_loss: 0.8292 | train_acc: 0.7099 | val_loss: 1.1834 | val_acc: 0.6039

